In [0]:

#Read ICD extracts from bronze layer
df=spark.read.format("parquet").load("/mnt/bronze/icd_codes/")

df.createOrReplaceTempView("staging_icd_codes")

In [0]:

%sql
CREATE TABLE IF NOT EXISTS silver.icd_codes (
    icd_code STRING,
    icd_code_type STRING,
    code_description STRING,
    inserted_date DATE,
    updated_date DATE,
    is_current_flag BOOLEAN
)

In [0]:

%sql
MERGE INTO
  silver.icd_codes AS target
USING
  staging_icd_codes AS source
ON target.icd_code = source.icd_code
WHEN MATCHED AND
  target.code_description != source.code_description
  THEN UPDATE SET
  target.code_description = source.code_description,
  target.updated_date = source.updated_date,
  target.is_current_flag = False
WHEN NOT MATCHED THEN INSERT (
    icd_code, icd_code_type, code_description, inserted_date, updated_date, is_current_flag
  )
  VALUES (
    source.icd_code,
    source.icd_code_type,
    source.code_description,
    source.inserted_date,
    source.updated_date,
    source.is_current_flag
  )